In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import re
import csv
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-classification-lenta/train_10.json
/kaggle/input/text-classification-lenta/emb_model.txt
/kaggle/input/text-classification-lenta/emb_eng.txt
/kaggle/input/text-classification-lenta/train_50.json
/kaggle/input/text-classification-lenta/train_10000.json
/kaggle/input/text-classification-lenta/submission_template.txt
/kaggle/input/text-classification-lenta/imdb_dataset.csv
/kaggle/input/text-classification-lenta/train_100.json
/kaggle/input/text-classification-lenta/train_5.json
/kaggle/input/text-classification-lenta/test.csv
/kaggle/input/text-classification-lenta/train_1000.json
/kaggle/input/text-classification-lenta/train_20.json
/kaggle/input/text-classification-lenta/train_5000.json


In [2]:
def data_preporation(filename):
    labels = []
    titles = []
    texts = []
    
    with open('../input/text-classification-lenta/' + filename) as f:
        templates = json.load(f)

    for section, commands in templates.items():
        labels.append(section)
        texts.append(commands['texts'])
        titles.append(commands['titles'])
   
    for i in range(len(titles)):
        for j in range(len(titles[i])):
            texts[i][j] += ' ' + titles[i][j]
            
    labels_dict = {'Экономика': 0, 'Спорт': 1, 'Культура': 2, 'Наука и техника': 3}

    tmp_labels = [labels_dict[label] for label in labels]

    correct_labels = []

    for i in range(len(texts)):
        for j in range(len(texts[i])):
            correct_labels.append(tmp_labels[i])

    preproc_texts = []
    
    for text in texts:
        for i in text:
            s = i.lower()
            s = re.sub("[^а-яА-Яa-zA-Z0-9]", " ", s)
            s = re.sub("\s+", " ", s)
            s = s.strip()
            preproc_texts += [s]
    
    tmp_dict = {'news': preproc_texts, 'labels': correct_labels}
    
    
    return pd.DataFrame(tmp_dict)

In [3]:
df = data_preporation('train_10000.json')
df

,news,labels
0,с 1 января 2000 года все телеканалы будут опла...,0
1,германский автопромышленный концерн volkswagen...,0
2,нераспределенная прибыль оао тюменнефтегаз доч...,0
3,две крупнейших телекоммуникационных компании с...,0
4,оао газ и нижегородский банк сбербанка россии ...,0
...,...,...
36254,компания microsoft добавила 84 наименования к ...,3
36255,российский космонавт юрий маленченко и астрона...,3
36256,опубликованы результаты самого подробного иссл...,3
36257,американский комедиант и актер арти лэнг artie...,3


In [4]:
news = df['news']
news

0        с 1 января 2000 года все телеканалы будут опла...
1        германский автопромышленный концерн volkswagen...
2        нераспределенная прибыль оао тюменнефтегаз доч...
3        две крупнейших телекоммуникационных компании с...
4        оао газ и нижегородский банк сбербанка россии ...
                               ...                        
36254    компания microsoft добавила 84 наименования к ...
36255    российский космонавт юрий маленченко и астрона...
36256    опубликованы результаты самого подробного иссл...
36257    американский комедиант и актер арти лэнг artie...
36258    сотовой связью во всем мире пользуются 3 3 мил...
Name: news, Length: 36259, dtype: object

In [5]:
nb_classes = 4
y_train = utils.to_categorical(df['labels'], nb_classes)
y_train

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [6]:
num_words = 5000
tokenizer = Tokenizer(num_words=num_words)

In [7]:
tokenizer.fit_on_texts(news)

In [8]:
sequences = tokenizer.texts_to_sequences(news)

In [9]:
index = 1
print(news[index])
print(sequences[index])

германский автопромышленный концерн volkswagen намерен полностью купить компанию skoda как стало сегодня известно из источников в штаб квартире volkswagen в вольфсбурге земля нижняя саксония германская сторона предложила правительству чехиипродать вольфсбургу 30 пакета акций skoda за 8 миллиардов крон 210 миллионов евро как сообщается чешские социал демократы пока заблокировали данную сделку считая коммерческое предложение volkswagen явно заниженным в то же время вольфсбург надеется что ему еще до конца года удастся выработать соглашение о приобретении чешской автокомпании в свою безраздельную собственность по мнению экспертов рыночная цена всего пакета акций skoda если учитывать бурное развитие компании в последние годы и спрос намашины как в западной так и восточной европе составляет более 800 миллионов евро volkswagen в 1991 году приобрел 70 ценных бумагskoda помимо чешского завода в северогерманскую автомобильную империю входят баварская фирма audi итальянская lamborgini британская

In [10]:
tokenizer.word_index['автопромышленный']

150497

In [11]:
max_news_len = 500
x_train = pad_sequences(sequences, maxlen=max_news_len)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D
model_cnn = Sequential()
model_cnn.add(Embedding(num_words, 32, input_length=max_news_len))
model_cnn.add(Conv1D(250, 5, padding='valid', activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dense(4, activation='softmax'))

In [13]:
model_cnn.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
conv1d (Conv1D)              (None, 496, 250)          40250     
_________________________________________________________________
global_max_pooling1d (Global (None, 250)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               32128     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 232,894
Trainable params: 232,894
Non-trainable params: 0
_________________________________________________________________


In [15]:
model_cnn_save_path = './best_model_cnn.h5'
checkpoint_callback_cnn = ModelCheckpoint(model_cnn_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [16]:
history_cnn = model_cnn.fit(x_train, 
                            y_train, 
                            epochs=5,
                            batch_size=128,
                            validation_split=0.1,
                            callbacks=[checkpoint_callback_cnn])

Epoch 1/5
255/255 [==============================] - 75s 289ms/step - loss: 0.8819 - accuracy: 0.6453 - val_loss: 0.3963 - val_accuracy: 0.8593

Epoch 00001: val_accuracy improved from -inf to 0.85935, saving model to ./best_model_cnn.h5
Epoch 2/5
255/255 [==============================] - 74s 292ms/step - loss: 0.0654 - accuracy: 0.9808 - val_loss: 0.2791 - val_accuracy: 0.8991

Epoch 00002: val_accuracy improved from 0.85935 to 0.89906, saving model to ./best_model_cnn.h5
Epoch 3/5
255/255 [==============================] - 75s 293ms/step - loss: 0.0247 - accuracy: 0.9938 - val_loss: 0.4436 - val_accuracy: 0.8682

Epoch 00003: val_accuracy did not improve from 0.89906
Epoch 4/5
255/255 [==============================] - 74s 292ms/step - loss: 0.0094 - accuracy: 0.9984 - val_loss: 0.4359 - val_accuracy: 0.8822

Epoch 00004: val_accuracy did not improve from 0.89906
Epoch 5/5
255/255 [==============================] - 74s 292ms/step - loss: 0.0028 - accuracy: 0.9998 - val_loss: 0.4533 

In [17]:
mtest_data = []

with open('../input/text-classification-lenta/test.csv', newline = '') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        mtest_data.append(row[1] + ' ' + row[0])

preproc_mtest_data = []

for text in mtest_data:
    s = text.lower()
    s = re.sub("[^а-яА-Яa-zA-Z0-9]", " ", s)
    s = re.sub("\s+", " ", s)
    s = s.strip()
    preproc_mtest_data += [s]

mtest_df = pd.DataFrame(preproc_mtest_data, columns = ['text'])
mtest_df

,text
0,россиянка елена дементьева в среду вышла в пол...
1,в новом фильме гая ричи под названием шерлок х...
2,европейский суд аннулировал решение еврокомисс...
3,авиакомпания lufthansa и группа ист лайн подпи...
4,национальное аэрокосмическое агентство сша nas...
...,...
412,американский авиастроительный концерн boeing п...
413,южная корея подала заявку на проведение чемпио...
414,президент россии владимир путин поручил правит...
415,форвард питерского зенита и сборной россии по ...


In [18]:
test_sequences = tokenizer.texts_to_sequences(mtest_df['text'])

In [19]:
x_test = pad_sequences(test_sequences, maxlen=max_news_len)
x_test

array([[   0,    0,    0, ..., 1704, 4930, 1288],
       [   0,    0,    0, ..., 1146, 3128,   11],
       [   0,    0,    0, ..., 4719,   31,  156],
       ...,
       [   0,    0,    0, ...,   84, 1673,   86],
       [   0,    0,    0, ...,  436,    1,  484],
       [   0,    0,    0, ...,   32,  489, 3766]], dtype=int32)

In [20]:
preds = model_cnn.predict(x_test)
np.argmax(preds[0])
list1 = []
for i in preds:
    list1.append(np.argmax(i))
    
labels_dict_reverse = {0: 'Экономика', 1: 'Спорт', 2: 'Культура', 3: 'Наукаитехника'}
answ = [labels_dict_reverse[label] for label in list1]


In [21]:
c = 2919
f = open('./sub.txt', 'w')
for i in range(len(answ)):
    f.write(str(c) + ',' + answ[i] + '\n')
    c += 1
f.close()